In [ ]:
from fileoperations.fileoperations import getFileNamesInDir
from symbtrdataextractor import symbtrreader, extractor
import os
import json
import pandas as pd 

import pdb

In [ ]:
symbTr = 'makam--form--usul--name--composer'

symbtrtxtfile = os.path.join('../txt', symbTr + '.txt')
symbtrmu2file = os.path.join('../mu2', symbTr + '.mu2')

usuldict = json.load(open('./unittests/data/usul_extended.json','r'))

savefile = os.path.join('..', '..', 'symbtr_corrections', 'txt', symbTr + '.txt')

In [ ]:
# extract symbtr data
txtdata = extractor.extract(symbtrtxtfile)[0]
mu2header = symbtrreader.readMu2Header(symbtrmu2file)[0]

# merge
data = extractor.merge(txtdata, mu2header)

In [ ]:
for usul in usuldict.values():
    for uv in usul['variants']:
        if uv['mu2_name'] == data['usul']['mu2_name']:
            mertebe = uv['mertebe']
            zaman = uv['num_pulses']
            break


In [ ]:
df = pd.read_csv(symbtrtxtfile, sep='\t')
for index, row in df.iterrows():
    # recompute the erroneous gracenotes with non-zero duration
    if row['Kod'] == 8 and row['Ms'] > 0:
        row['Pay'] = 0
        row['Payda'] = 0
        row['Ms'] = 0
        
    # recompute zaman and mertebe, if we hit kod 51
    if row['Kod'] == 51:
        zaman = row['Pay']
        mertebe = row['Payda']
        offset_incr = 0
    else:
        # compute offset
        offset_incr = 0 if row['Payda'] ==0 else float(row['Pay'])/row['Payda']*mertebe/zaman
    if index == 0:
        row['Offset'] = offset_incr
    else:
        prev_row = df.iloc[index-1]
        row['Offset'] = offset_incr + prev_row['Offset']
                
    # change null to empty string
    for key, val in row.iteritems():
        if pd.isnull(val):
            row[key] = ''
            
    # make sure that "Sira" column continues consecutively
    row['Sira'] = index+1
    
    # reassign
    df.iloc[index] = row
    

In [ ]:
df.to_csv(savefile, sep='\t', index=False)

In [ ]:
txtdata = extractor.extract(savefile)[0]
import numbers
if not (round(row['Offset']*10000)*0.0001).is_integer():
    print "Ends prematurely!"